In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import datetime as dt
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv') # Загружаем данные
data.sample(5)                        # Просмотр 5 случайных образцов

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
600,tt0910970,180000000,521311860,WALLÂ·E,Ben Burtt|Elissa Knight|Jeff Garlin|Fred Willa...,Andrew Stanton,An adventure beyond the ordinar-E.,WALLÂ·E is the last robot left on an Earth tha...,98,Animation|Family,Walt Disney Pictures|Pixar Animation Studios,6/22/2008,7.6,2008
1044,tt1615065,45000000,47000000,Savages,Blake Lively|Taylor Kitsch|Aaron Taylor-Johnso...,Oliver Stone,Young Beautiful Deadly.,Pot growers Ben and Chon face off against the ...,131,Crime|Drama|Thriller,Ixtlan|Relativity Media|Onda Entertainment,7/6/2012,6.2,2012
1690,tt0413267,160000000,798958165,Shrek the Third,Mike Myers|Eddie Murphy|Cameron Diaz|Julie And...,Chris Miller|Raman Hui,Who's ready for Thirds?,The King of Far Far Away has died and Shrek an...,93,Fantasy|Adventure|Animation|Comedy|Family,DreamWorks Animation,5/17/2007,5.8,2007
1758,tt0478134,23000000,29541790,In the Valley of Elah,Tommy Lee Jones|Charlize Theron|Susan Sarandon...,Paul Haggis,Sometimes finding the truth is easier than fac...,A career officer and his wife work with a poli...,124,History|Drama|Thriller|Crime|Mystery,Summit Entertainment|Blackfriars Bridge Films|...,8/31/2007,6.2,2007
525,tt0268397,25000000,80936232,Jimmy Neutron: Boy Genius,Debi Derryberry|Megan Cavanagh|Mark DeCarlo|Je...,John A. Davis,"He may be small, but he's got a big brain!",Jimmy Neutron is a boy genius and way ahead of...,83,Adventure|Fantasy|Animation|Action|Comedy,Nickelodeon Movies|Universal Cartoon Studios|P...,12/21/2001,5.5,2001


# Предобработка

In [3]:
answers = {} # Создадим словарь для ответов

data['profit'] = data.revenue - data.budget # Создадим новый столбец - прибыль

def my_sort(x):
    '''Функция возвращает отсортированный список'''
    x.sort()
    return x

In [4]:
data.describe() # Очень полезно знать характеристики полей

,budget,revenue,runtime,vote_average,release_year,profit
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000,1.889000e+03
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773,1.010545e+08
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841,1.830733e+08
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000,-1.657101e+08
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000,5.000000e+06
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000,4.400041e+07
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000,1.201842e+08
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000,2.544506e+09


In [5]:
data.info() # Также полезно знать формат данных DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
 14  profit                1889 non-null   int64  
dtypes: float64(1), int64(

In [6]:
data.sample(5) # Просмотр 5 случайных образцов обновлённого DataFrame

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
827,tt1124035,12500000,75993061,The Ides of March,Ryan Gosling|George Clooney|Philip Seymour Hof...,George Clooney,Ambition seduces. Power corrupts.,Dirty tricks stand to soil an ambitious young ...,101,Drama,Appian Way|Crystal City Entertainment|Cross Cr...,9/24/2011,6.4,2011,63493061
1019,tt1646987,150000000,301000000,Wrath of the Titans,Sam Worthington|Liam Neeson|Ralph Fiennes|Rosa...,Jonathan Liebesman,Feel the Wrath,A decade after his heroic defeat of the monstr...,99,Adventure,Legendary Pictures|Thunder Road Pictures|Warne...,3/27/2012,5.5,2012,151000000
716,tt0410377,37000000,100076342,Nim's Island,Abigail Breslin|Jodie Foster|Gerard Butler|Mic...,Jennifer Flackett|Mark Levin,Your adventure starts here.,A young girl inhabits an isolated island with ...,96,Adventure|Comedy|Family|Fantasy,Walden Media,4/3/2008,5.5,2008,63076342
1732,tt0423294,85000000,149044513,Surf's Up,Shia LaBeouf|Jeff Bridges|Zooey Deschanel|Jon ...,Ash Brannon|Chris Buck,"In the coldest place on Earth, he's the hottes...",Cody is a surfing penguin from Shiverpool who ...,85,Animation|Comedy|Family,Sony Pictures Animation,6/8/2007,5.9,2007,64044513
725,tt0800369,150000000,444115007,Thor,Chris Hemsworth|Natalie Portman|Tom Hiddleston...,Kenneth Branagh,Two worlds. One hero.,"Against his father Odin's will, The Mighty Tho...",115,Adventure|Fantasy|Action,Marvel Studios|Marvel Entertainment,4/21/2011,6.5,2011,294115007


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

*Но если я получил их честно, то считаю, что я могу*
*заполнить словарь правильным ответом :)*

In [7]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'
# +
# В первом варианте использовалось стандартные подходы фильтрации:
#     сравнение поля "budget" с максимальном значением по столбцу
# Во втором варианте использовалась полезная функция idxmax()

# P.S. Не смотрел..

ВАРИАНТ 1

In [8]:
# Ключевой момент: фильтрация по максимальному бюджету
data[data.budget == data.budget.max()].original_title.iloc[0]

'Pirates of the Caribbean: On Stranger Tides'

ВАРИАНТ 2

In [9]:
# Ключевой момент: функция idxmax()
data.loc[data.budget.idxmax()].original_title

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [10]:
answers['2'] = 'Gods and Generals (tt0279111)'
# +
# По аналогии в первом варианте использовалось
# стандартные подходы фильтрации:
#     сравнение поля "budget" с максимальном значением по столбцу
# А во втором варианте использовалась полезная функция idxmax()

# P.S. Аналогично, не смотрел..

ВАРИАНТ 1

In [11]:
# Ключевой момент: фильтрация по максимальной продолжительности
data[data.runtime == data.runtime.max()].original_title.iloc[0]

'Gods and Generals'

ВАРИАНТ 2

In [12]:
# Ключевой момент: функция idxmax()
data.loc[data.runtime.idxmax()].original_title

'Gods and Generals'

# 3. Какой из фильмов самый короткий (в минутах)?





In [13]:
answers['3'] = 'Winnie the Pooh (tt1449283)'
# +
# Аналогично задачам 1, 2

# P.S. Тоже не смотрел этот мультфильм

ВАРИАНТ 1

In [14]:
# Ключевой момент: фильтрация по минимальной продолжительности
data[data.runtime == data.runtime.min()].original_title.iloc[0]

'Winnie the Pooh'

ВАРИАНТ 2

In [15]:
# Ключевой момент: функция idxmin()
data.loc[data.runtime.idxmin()].original_title

'Winnie the Pooh'

# 4. Какова средняя длительность фильмов?


In [16]:
answers['4'] = 110
# +
# Для решения задачи используется агрегирующая функция mean()

# P.S. По ощущениям так и есть.

In [17]:
# Ключевой момент: функция вычисления среднего mean()
round(data.runtime.mean()) # Для округления используем round()

110

# 5. Каково медианное значение длительности фильмов? 

In [18]:
answers['5'] = 107
# +
# Для решения задачи используется агрегирующая функция median()
# round() использовался для получения целого типа

# P.S. Слабо отличается от средненего..

In [19]:
# Ключевой момент: функция вычисления среднего медиального median()
round(data.runtime.median()) # Для округления используем round()

107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [20]:
answers['6'] = 'Avatar (tt0499549)'
# +
# Решается по аналогии с предыдущими задачами

# P.S. Смотрел предвзято, много позже после выхода
# сюжет избит, спецэффекты на высоте, но не цепляет,
# как в детстве Т800; и склонен думать, что ГГ - предатель :)

ВАРИАНТ 1

In [21]:
# Ключевой момент: фильтрация по максимальной прибыли
data[data.profit == data.profit.max()].original_title.iloc[0]

'Avatar'

ВАРИАНТ 2

In [22]:
# Ключевой момент: функция idxmax()
data.loc[data.profit.idxmax()].original_title

'Avatar'

# 7. Какой фильм самый убыточный? 

In [23]:
answers['7'] = 'The Lone Ranger (tt1210819)'
# +
# Решается по аналогии с предыдущими задачами

# P.S. Не смотрел и этот фильм..

ВАРИАНТ 1

In [24]:
# Ключевой момент: фильтрация по минимальной прибыли
data[data.profit == data.profit.min()].original_title.iloc[0]

'The Lone Ranger'

ВАРИАНТ 2

In [25]:
# Ключевой момент: функция idxmin()
data.loc[data.profit.idxmin()].original_title

'The Lone Ranger'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [26]:
answers['8'] = 1478
# +
# Вариант 1 путём фильтрации, а затем вызывается
# агрегирующая функия count() для одного из столбцов
# Вариант 2 некое велосипедостроение, скорее показывает,
# что штатные функции Python-a всё ещё strong

# P.S. Это ~80%

ВАРИАНТ 1

In [27]:
# Ключевой момент: фильтрация по фильмам, где кассовые сборы больше бюджета
data[data.revenue > data.budget]['imdb_id'].count()

1478

ВАРИАНТ 2

In [28]:
# Ключевой момент: фильтрация по фильмам, где кассовые сборы больше бюджета
len(data[data.revenue > data.budget])

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [29]:
answers['9'] = 'The Dark Knight (tt0468569)'
# +
# Первый вариант использует сортировку по столбцу "revenue"
# Во втором варианте использовался старый подход (1, 2,..),
# но для разнообразия использовал столбец "release_date"

# P.S. фильм смотрел, показался нудным; как сказал Д. Пучков
# всю соль убили переводчики. И чтоб насладиться им, надо
# смотреть в оригинале, но на слух я вообще не понимаю :(

ВАРИАНТ 1

In [30]:
# Ключевой момент: фильтрация+сортировка
data.loc[data.release_year.isin([2008])].sort_values(by='revenue', ascending=False)['original_title'].iloc[0]

'The Dark Knight'

In [31]:
# Ключевой момент: фильтрация+idxmax()
data_2008 = data[data.release_date.str.contains('2008')].copy() # Копируем отфильтрованный DataFrame
data_2008.loc[data_2008.revenue.idxmax()].original_title        # Дальше по аналогии с задачами 1, 2,..

'The Dark Knight'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [32]:
answers['10'] = 'The Lone Ranger'
# +
# Решается по аналогии с предыдущими задачей 9
# Можно было отфильтровать, как в 1, 2,..

# P.S. Понятия не имею о чём он..

In [33]:
# Ключевой момент: фильтрация+сортировка
data.loc[data.release_year.isin([2012, 2013, 2014])].sort_values(by='profit')['original_title'].iloc[0]

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [34]:
answers['11'] = 'Drama'
# +
# Два подхода, один стандартный, второй со сводной таблицей
# Дополнительно модифицируем столбец "genres" функцией str.split()

# P.S. Никогда не задумывался об этом

ВАРИАНТ 1

In [35]:
# Ключевой момент: модификация и раскрытие столбца
data.genres.str.split('|').explode().value_counts().index[0] # В одну строчку

'Drama'

ВАРИАНТ 2

In [36]:
# Создаём копию DataFrame
genres_data = data.copy()

# Обновляем поле "genres", раскрывая его посредством функции explode()
genres_data.genres = data.genres.str.split('|').explode('genres')

# Создаём сводную таблицу: в строках жанры, в столбцах идентификаторы фильмов, в качестве значений название фильмов
genres_pivot = genres_data.pivot_table(index = 'genres', columns = 'imdb_id', values = 'original_title', aggfunc = 'count', fill_value = 0)

# Выбираем максимальную сумму по строкам
genres_pivot.sum(axis=1).idxmax()

'Drama'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [37]:
answers['12'] = 'Drama'
# +
# Уже стандартные подходы, но во втором вспомнил, что есть столбец "profit"

# P.S. Ok. Буду знать.

ВАРИАНТ 1

In [38]:
# Ключевой момент: фильтрация и раскрытие столбца
data[data.revenue > data.budget].genres.str.split('|').explode().value_counts().index[0]

'Drama'

ВАРИАНТ 2

In [39]:
# Ключевой момент: фильтрация и раскрытие столбца
data[data.profit > 0].genres.str.split('|').explode().value_counts().index[0]

'Drama'

ВАРИАНТ 3

In [40]:
# Создаём DataFrame bb_data (сокр. blockbuster_data)
bb_data = data.query('profit > 0').copy()

# Преобразуем bb_data в Series, используя функции, split() и explode()
bb_data = bb_data.genres.str.split('|').explode()

# Создаём сумму по возрастанию и выводим первый элемент
bb_data.value_counts().index[0]

'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [41]:
answers['13'] = 'Peter Jackson'
# +
# Важно разбить столбец по режиссёрам с помощью str.split('|')
# В первом варианте группируем по "director", а дальше выполняем стандартные
# относительно столбца "revenue"
# Во втором случае очень полезной оказалась сводная таблица и параметр axis
# позволивший сделать суммирование по строкам

# P.S. Чего только Властилин Колец стоит. В переводе Д.Пучкова веселей :)

ВАРИАНТ 1

In [42]:
# Копируем данные
director_data = data.copy()

# Разбиваем содержимое столбца "director"
director_data.director = director_data.director.str.split('|')

# Раскрываем его
director_data = director_data.explode('director')

# Группируем, суммируем, сортируем
director_data.groupby('director').revenue.sum().sort_values(ascending=False).index[0]

'Peter Jackson'

ВАРИАНТ 2

In [43]:
# Копируем данные
director_data = data.copy()

# Разбиваем содержимое столбца "director"
director_data.director = director_data.director.str.split('|')

# Раскрываем его
director_data = director_data.explode('director')

# Создаём сводную таблицу: в строках ФИО, в столбцах идентификаторы фильмов, в качестве значений сборы
director_pivot = director_data.pivot_table(index='director', columns='imdb_id', values='revenue', aggfunc='sum', fill_value=0)

# Выбираем максимальную сумму по строкам
director_pivot.sum(axis=1).idxmax()

'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [44]:
answers['14'] = 'Robert Rodriguez'
# +
# Все 3 варианта дают идентичные ответы..
# Даже если непосредственно отфильтровать по Бэю, то у него 7 картин
# Но в силу того, что ни Бэя, ни Андерсона нет в вариантах ответа, то
# выбираем какого-то Родригеса.

# P.S. Согласен с комментарием в поле ответов - цифры говорят другое.

ВАРИАНТ 1

In [45]:
# Ключевой момент: корректная фильтрация
data[data.genres.str.contains('Action')].director.value_counts()

Michael Bay              7
Paul W.S. Anderson       7
Antoine Fuqua            6
Robert Rodriguez         6
Ridley Scott             6
                        ..
Tommy Wirkola            1
Christian Alvart         1
Sean McNamara            1
Jay Russell              1
Gina Prince-Bythewood    1
Name: director, Length: 345, dtype: int64

ВАРИАНТ 2

In [46]:
# Создаём новый DataFrame
action_data = data[data.genres.str.contains('Action')].copy()

# Создаём сводную таблицу: в строках ФИО, в столбцах идентификаторы фильмов, в качестве значений сборы
action_pivot = action_data.pivot_table(index='director', columns='imdb_id', values='genres', aggfunc='count', fill_value=0)
#display(action_pivot)

# Выбираем максимальную сумму по строкам
action_pivot.sum(axis=1).sort_values(ascending=False)

director
Michael Bay           7
Paul W.S. Anderson    7
Robert Rodriguez      6
Ridley Scott          6
Antoine Fuqua         6
                     ..
Les Mayfield          1
Kurt Wimmer           1
Kevin Smith           1
Kevin Macdonald       1
Adam McKay            1
Length: 345, dtype: int64

ВАРИАНТ 3

In [47]:
# Ключевой момент: фильтрация, группировка и сортировка
data[data.genres.str.contains('Action')].groupby(by='director').imdb_id.count().sort_values(ascending=False)

director
Michael Bay           7
Paul W.S. Anderson    7
Robert Rodriguez      6
Ridley Scott          6
Antoine Fuqua         6
                     ..
Les Mayfield          1
Kurt Wimmer           1
Kevin Smith           1
Kevin Macdonald       1
Adam McKay            1
Name: imdb_id, Length: 345, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [48]:
answers['15'] = 'Chris Hemsworth'
# +
# Первый вариант стандартным путём решался
# Второй вариант с помощью сводной таблицы

# P.S. Кто эти люди?..

ВАРИАНТ 1

In [49]:
# Копируем DataFrame
actor_data = data.copy()

# Фильтруем
actor_data = actor_data[actor_data.release_year == 2012]

# Обновляем поле "cast", фильтруя и раскрывая его посредством функции explode()
actor_data.cast = actor_data.cast.str.split('|')
actor_data = actor_data.explode('cast')

# Группируем по актёрам, суммируем по сборам и сортируем
actor_data.groupby('cast').revenue.sum().sort_values(ascending=False).index[0]

'Chris Hemsworth'

ВАРИАНТ 2

In [50]:
# Создаём копию исходного DataFrame
best_actor = data.copy()

# Обновляем поле "cast", фильтруя и раскрывая его посредством функции explode()
best_actor.cast = data[data.release_year == 2012].cast.str.split('|')
best_actor = best_actor.explode('cast')

# Создаём сводную таблицу в качестве строк ФИО, в качестве столбцов идентификатор, в качестве значений сборы
best_pivot = best_actor.pivot_table(index='cast', columns='imdb_id', values='revenue', fill_value=0)
best_pivot.sum(axis=1).sort_values(ascending=False).index[0]

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [51]:
answers['16'] = 'Matt Damon'
# +
# Использовался стандартный подход: фильтрация, модификация, эксплоуд..

# P.S. Особенно понравился в Отступниках, ну и картошка на Марсе. Борн меньше)

In [52]:
# Ключевой момент: фильтрация и раскрытие модифицированного столбца
data[data.budget>data.budget.mean()]['cast'].str.split('|').explode().value_counts().index[0]

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [53]:
answers['17'] = 'Action'
# +
# Использовался стандартный подход: фильтрация, модификация, эксплоуд..

# P.S. Я не удивлён.

In [54]:
# Ключевой момент: фильтрация и раскрытие модифицированного столбца
data[data.cast.str.contains('Nicolas Cage')]['genres'].str.split('|').explode().value_counts().index[0]

'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [55]:
answers['18'] = 'K-19: The Widowmaker'
# +
# Фильтрация, сортировка, ..
# Люблю, когда в одну строчку

# P.S. В целом достойно наших показали, но нудно.

In [56]:
# Ключевой момент: фильтрация и сортировка
data[data.production_companies.str.contains('Paramount Pictures')].sort_values(by='profit')['original_title'].iloc[0]

'K-19: The Widowmaker'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [57]:
answers['19'] = 2015
# +
# Группировка, суммирование по сборам, сортировка

# P.S. Спорить не буду

In [58]:
# Ключевой момент: группировка и сортировка
data.groupby(by='release_year').revenue.sum().sort_values(ascending=False).index[0]

2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [59]:
answers['20'] = 2014
# +
# Фильтрация, группировка, суммирование по сборам, сортировка

# P.S. Я не против

In [60]:
# Ключевой момент: фильтрация, группировка и сортировка
data[data.production_companies.str.contains('Warner Bros')].groupby(by='release_year').profit.sum().sort_values(ascending=False).index[0]

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [61]:
answers['21'] = 'September'
# +
# Используется преобразование для даты, затем выделяется месяц и value_counts()
# Испльзую последовательно lambda функцию

In [62]:
# Ключевой момент: приведение к нужному формату данных
data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%B')).value_counts().index[0]

'September'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [63]:
answers['22'] = 450
# +
# Фильтрация, дальше стандартная функция nunique()
# Также использовалась полезная lambda функция

# P.S. Почти четверть датасета. Полагаю всё поровну для всех времён года

In [64]:
# Ключевой момент: фильтрация с приведением к нужному формату данных
data[data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%m') in ['06', '07', '08'])].imdb_id.nunique()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [65]:
answers['23'] = 'Peter Jackson'
# +
# В первом варианте использовался стандартный подход: фильтрация и value_counts()
# Во втором и третьем - разные вариации на тему lambda функции

# P.S. Спорный вопрос, он Юж. Зеландии, там зимой - лето :)

ВАРИАНТ 1

In [66]:
# Ключевой момент: фильтрация с приведением к нужному формату данных
data[data.release_date.apply(pd.to_datetime).dt.month.isin([1, 2, 12,])].director.value_counts().index[0]

'Peter Jackson'

ВАРИАНТ 2

In [67]:
# Ключевой момент: фильтрация с приведением к нужному формату данных
data[data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%m') in ['01', '02', '12'])].director.value_counts().index[0]

'Peter Jackson'

ВАРИАНТ 3

In [68]:
# Ключевой момент: фильтрация с приведением к нужному формату данных
data[data.release_date.apply(pd.to_datetime).apply(lambda x: x.strftime('%B') in ["December", "January", "February"])].director.value_counts().index[0]

'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [69]:
answers['24'] = 'Four By Two Productions'
# +
# Оставляем списко как есть, так как может быть несколько компаний, а в ответе одна

# P.S. Что они хотя бы сняли?

In [70]:
# Копируем DataFrame
studio_data = data.copy()

# Модифицируем содержимое "original_title"
studio_data.original_title = studio_data.original_title.str.len() # Считаем, что пробел тоже символ

# Разбиваем и раскрываем по "production_companies"
studio_data.production_companies = studio_data.production_companies.str.split('|')
studio_data = studio_data.explode('production_companies')

# Группируем по компаниям, выбираем максимумы и сортируем
studio_data.groupby(by='production_companies').max().original_title.sort_values(ascending=False)

production_companies
Four By Two Productions                   83
Twentieth Century Fox Film Corporation    83
Walden Media                              62
Walt Disney                               62
Jim Henson Company, The                   59
                                          ..
Everest Entertainment                      3
Berlanti Productions                       3
XM2 Productions                            2
Global Entertainment Group                 2
Ixtlan Productions                         2
Name: original_title, Length: 1771, dtype: int64

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [71]:
answers['25'] = 'Midnight Picture Show'
# +
# Использовался стандартный подход с lambda функциями в первом варианте
# Во втором - сводная таблица
# Поскольку overview_data больше нигде не используется, решил оставить его в данной ячейке

# P.S. Даже не слышал о такой

ВАРИАНТ 1

In [72]:
# Копируем DataFrame
word_data = data.copy()

# Модифицируем содержимое "overview"
word_data.overview = word_data.overview.apply(lambda x: len(x.split()))

# Разбиваем и раскрываем по "production_companies"
word_data.production_companies = word_data.production_companies.str.split('|')
word_data = word_data.explode('production_companies')

# Группируем по компаниям вычисляем среднее и выводим нужную компанию
word_data.groupby(['production_companies'])['overview'].mean().idxmax()

'Midnight Picture Show'

ВАРИАНТ 2

In [73]:
# Копируем DataFrame
overview_data = data.copy()

# Создаём столбец "overview_len"
overview_data['overview_len'] = overview_data.overview.apply(lambda x:len(x.split(' ')))

# Разбиваем и раскрываем по "production_companies"
overview_data['production_companies'] = overview_data.production_companies.str.split('|')
overview_data = overview_data.explode('production_companies')

# Создаём сводную таблицу и сортируем по "overview_len"
overview_data.pivot_table(['overview_len'],['production_companies'], aggfunc='mean').sort_values(by='overview_len', ascending=False).index[0]

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [74]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'
# +
# Фильтрация, далее агрегирующая функция quantile()
# Выбор ответа был обусловлен предложенными вариантами и полученной выборкой

# P.S. Будет время посмотрю, может и Дарк Найта с субтитрами огорю

In [75]:
# Ключевой момент: фильтрация и функция quantile()
data[data['vote_average'] > data['vote_average'].quantile(q=0.99)]['original_title']

9                                          Inside Out
34                                               Room
118                                      Interstellar
119                           Guardians of the Galaxy
125                                The Imitation Game
128                                         Gone Girl
138                          The Grand Budapest Hotel
370                                         Inception
599                                   The Dark Knight
872                                       The Pianist
1081    The Lord of the Rings: The Return of the King
1183                          The Wolf of Wall Street
1191                                 12 Years a Slave
1800                                          Memento
Name: original_title, dtype: object

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [76]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint'
# +
# Используется библиотека itertools

# Ключевой момент для решения задачи - предварительная сортировка по актёрам,
# чтобы избежать получения ситуации, когда будут встречаться записи вроде
# "Л. Гурченко, Л. Куравлёв" и "Л. Куравлёв, Л. Гурченко", что с точки зрения кода - 
# разные данные. Следовательно, дальнейшие вычисления будут ошибочными.

# Вариант 1 предполагает модификацию поля, соответствующего актёрам, далее
# сортировка актёров, затем создание пар, затем эксплоуд и подсчёт
# В данном случае проведён трюк, где lambda функция неявно выполняет 2 действия:
# lambda x: [x.sort(), list(itertools.combinations(x, 2))][1] - сначала сортирует,
# а затем создаёт комбинации. На выход выдаёт элемент с индексом 1 - то что нужно!
# Второй подход ничем не отличается от предыдущего, но в нём я решил использовать
# хотя бы одну личную функцию на весь объём заданий.
# Выбор ответа был обусловлен предложенными вариантами и полученной выборкой

# P.S. Если не делать сортировку, то третье место займёт пара Rupert Grint+Emma Watson,
#      которые снялись вместе "почему-то" (см. выше) только в 7 фильмах о Гарри Поттере..
#      Но мы то знаем ;)

ВАРИАНТ 1

In [77]:
# Ключевой момент: корректная сортировка по актёрам и раскрытие модифицированного столбца
data.cast.str.split('|').apply(lambda x: [x.sort(), list(itertools.combinations(x, 2))][1]).explode().value_counts()

(Daniel Radcliffe, Rupert Grint)       8
(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Emma Watson)        8
(Ben Stiller, Owen Wilson)             6
(Helena Bonham Carter, Johnny Depp)    6
                                      ..
(Jason Statham, Miranda Hart)          1
(Dustin Hoffman, Freddie Highmore)     1
(Dustin Milligan, Joel David Moore)    1
(Gena Rowlands, James Garner)          1
(Eddie Murphy, Jennifer Hudson)        1
Name: cast, Length: 17943, dtype: int64

ВАРИАНТ 2

In [78]:
# Ключевой момент: корректная сортировка по актёрам и раскрытие модифицированного столбца
data.cast.str.split('|').apply(my_sort).apply(lambda x: list(itertools.combinations(x, 2))).explode().value_counts()

(Daniel Radcliffe, Rupert Grint)       8
(Emma Watson, Rupert Grint)            8
(Daniel Radcliffe, Emma Watson)        8
(Ben Stiller, Owen Wilson)             6
(Helena Bonham Carter, Johnny Depp)    6
                                      ..
(Jason Statham, Miranda Hart)          1
(Dustin Hoffman, Freddie Highmore)     1
(Dustin Milligan, Joel David Moore)    1
(Gena Rowlands, James Garner)          1
(Eddie Murphy, Jennifer Hudson)        1
Name: cast, Length: 17943, dtype: int64

# Submission

In [79]:
# В конце проверяю свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 'September',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [80]:
# Чтобы убедиться, что ничего не пропустил
len(answers)

27

## В целом считаю, что решил все задания, используя стандартные возможности Python&Pandas. Одна функция не в счёт, так как я внёс её умышленно, чтоб была :)

## Спасибо за Внимание!